<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Teorema de la falsa posición</h2>
        <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/False_position_method.svg/702px-False_position_method.svg.png alt=middle style="height: 340px">
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [29]:
format long

function [xk,e]=false_pos_aux(footion, ck, xk, tol, i, iter, error, e)
    if footion(xk) * footion(ck) < 0
        if i >= iter || error <= tol
            xk=xk;
            e=e;
        else 
            x2 = xk - ((xk - ck) / (footion(xk) - footion(ck))) * footion(xk);
            error = abs(x2-xk);
            e = [e error];
            xk = false_pos_aux(footion, ck, x2, tol, i+1, iter, error, e);
        end
    else
        xk = 0;
        e = [];
    end
end


function [xk,e]=false_pos(f, x0, x1, tol, iter)
    footion = str2func(strcat('@(x)',f));
    if footion(x0) * footion(x1) < 0 && tol > 10^-12
        x2 = x1 - ((x1 - x0) / (footion(x1) - footion(x0))) * footion(x1);
        error = abs(x2-x1);
        [a,e] = false_pos_aux(footion, x0, x2, tol, 0, iter, error, [error]);
        [b,e] = false_pos_aux(footion, x1, x2, tol, 0, iter, error, [error]);
        if a
            xk = a;
            e = e;
        else
            xk = b;
            e = e;
        end
    else
        display("error");
    end
end

[xk,e]=false_pos('cos(x)-x', 1/2, pi/4, 10^-2, 10)

xk =    7.390581392138897e-01
e =

   4.901402456086612e-02   2.674000377307584e-03

